<a href="https://colab.research.google.com/github/vrra/FGAN-Build-a-thon/blob/main/Notebooks2023/Read-semi-annotated-push-to-argilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created: 3 Jan 2024

Aaron, Othniel, Vishnu.

Modification History: 4 Jan 2024: Aaron, Frank, Othniel, Vishnu: Changed the data schema to a simpler format. Bye-bye "for_supervised_fine_tuning" format.

Description:

This notebook pulls records from HF hub (semi annotated dataset) and pushes them to HF spaces argilla (for 100% annotation).

Pre-requisites:

the following notebooks are already run:

1. Create the raw dataset in HF hub.

2. Configure the argilla dataset

3. add records in the argilla dataset from the raw dataset

4. partially annotate the dataset in UI - offline

5. Save the annotated dataset into HF hub

Finally This notebook pulls records from HF hub (semi annotated dataset) and pushes them to HF spaces argilla (for 100% annotation).

## Install Libraries

Install the latest version of Argilla in Colab, along with other libraries and models used in this notebook.

In [1]:
!pip install argilla datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


Prerequisites

Deploy Argilla Server on [HF Spaces](https://huggingface.co/new-space?template=argilla/argilla-template-space).


More info on Installation [here](../getting_started/installation/deployments/deployments.html).

## Secretes needed




* `ARGILLA_API_URL`: It is the url of the Argilla Server.
  * If you're using HF Spaces, it is constructed as `https://[your-owner-name]-[your_space_name].hf.space`.
* `ARGILLA_API_KEY`: It is the API key of the Argilla Server. It is `owner` by default.
* `HF_TOKEN`: It is the Hugging Face API token. It is only needed if you're using a [private HF Space](https://docs.argilla.io/en/latest/getting_started/installation/deployments/huggingface-spaces.html#deploy-argilla-on-spaces). You can configure it in your profile: [Setting > Access Tokens](https://huggingface.co/settings/tokens).
* `workspace`: admin


In [2]:
import argilla as rg
from argilla._constants import DEFAULT_API_KEY

In [3]:
from google.colab import userdata
api_url= userdata.get('my_argilla_url')
api_key= userdata.get('my_argilla_key')

import argilla as rg
rg.init(api_url=api_url, api_key=api_key)

# # If you want to use your private HF Space
# rg.init(extra_headers={"Authorization": f"Bearer {hf_token}"})

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:168: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(


In [99]:
from datasets import load_dataset

# Load and inspect a semi annotated dataset from the Hugging Face Hub
# (and not the pre-processed or annotated dataset in the spaces).
# vishnuramov/itu_annotated_dataset is the semi annotated dataset name in HF hub
# (and not annotated dataset in the spaces nor the raw dataset in the HF Hub)
hf_dataset = load_dataset('vishnuramov/itu_annotated_dataset')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [82]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['background', 'prompt', 'response', 'response_correction', 'response_correction-suggestion', 'response_correction-suggestion-metadata', 'external_id', 'metadata'],
        num_rows: 98
    })
})

In [100]:
custom_dataset = rg.FeedbackDataset.from_argilla(name="fgan-annotate-dataset", workspace="admin")
records = [
    rg.FeedbackRecord(
        fields={"background": record["background"],
                "prompt": record["prompt"],
                "response": record["response"],
                }
    )
    for record in hf_dataset['train']
    ]
custom_dataset

RemoteFeedbackDataset(
   id=12bbed5e-35ce-46a5-9613-a98f893b830e
   name=fgan-annotate-dataset
   workspace=Workspace(id=6196e1fe-7cc5-4ef4-b608-d98a8bc8fbc8, name=admin, inserted_at=2024-01-02 14:17:38.856061, updated_at=2024-01-02 14:17:38.856061)
   url=https://vishnuramov-itu-t-build-a-thon.hf.space/dataset/12bbed5e-35ce-46a5-9613-a98f893b830e/annotation-mode
   fields=[RemoteTextField(id=UUID('7ccfdb4c-dbd3-472e-a5b4-399839a754a8'), client=None, name='background', title='Background', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('305e73c2-455b-4eb6-ad79-3c6c6184fc17'), client=None, name='prompt', title='Prompt', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('3312f16a-8eb3-4381-a852-066b43a0f9c8'), client=None, name='response', title='Final Response', required=True, type='text', use_markdown=False)]
   questions=[RemoteTextQuestion(id=UUID('c590fe5a-bccc-4e06-872b-a2b75bad38e4'), client=None, name='response_correction', title='

In [101]:
def extract_background_prompt_response(str_text: str) -> Dict[str, Any]:
    '''Extract the anthropic prompt from a prompt and response pair.'''
    background_prompt = str_text.lower().find("background:")
    start_prompt = str_text.lower().find("<human>:")
    end_prompt = str_text.lower().rfind("<bot>:")

    if (background_prompt != -1) and (start_prompt == -1 ) and (end_prompt == -1):
      #only background is present
      background = str_text[background_prompt:].strip()
      prompt = ""
      response = ""
    elif (background_prompt == -1) and (start_prompt != -1 ) and (end_prompt == -1):
      #only human is present
      background = ""
      prompt = str_text[start_prompt:].strip()
      response = ""
    elif (background_prompt == -1) and (start_prompt == -1 ) and (end_prompt != -1):
      #only bot is present
      background = ""
      prompt = ""
      response = str_text[end_prompt:].strip()

    elif (background_prompt != -1) and (start_prompt != -1 ) and (end_prompt == -1):
      #only background and human are present
      background = str_text[background_prompt:start_prompt].strip()
      prompt = str_text[start_prompt:].strip()
      response = ""
    elif (background_prompt != -1) and (start_prompt == -1 ) and (end_prompt != -1):
      #only background and bot are present
      background = str_text[background_prompt:end_prompt].strip()
      prompt = ""
      response = str_text[end_prompt:].strip()
    elif (background_prompt == -1) and (start_prompt != -1 ) and (end_prompt != -1):
      #only human and bot are present
      background = ""
      prompt = str_text[start_prompt:end_prompt].strip()
      response = str_text[end_prompt:].strip()
    else:
      #all 3 are present
      background = str_text[background_prompt:start_prompt].strip()
      prompt = str_text[start_prompt:end_prompt].strip()
      response = str_text[end_prompt:].strip()

    return {"background": background, "prompt": prompt, "response": response}

In [102]:
for i, record in enumerate(hf_dataset['train']):
    if (len(record['response_correction'])):
      bg=extract_background_prompt_response(record['response_correction'][0]['value'])['background']
      if (len(bg)):
        records[i].fields['background'] = bg
      else:
        records[i].fields['background'] = record['background']
      pr=extract_background_prompt_response(record['response_correction'][0]['value'])['prompt']
      if (len(pr)):
        records[i].fields['prompt'] = pr
      else:
        records[i].fields['prompt'] = record['prompt']
      rc=extract_background_prompt_response(record['response_correction'][0]['value'])['response']
      if (len(rc)):
        records[i].fields['response'] = rc
      else:
        records[i].fields['response'] = record['response']
    else:
      records[i].fields['background'] = record['background']
      records[i].fields['prompt'] = record['prompt']
      records[i].fields['response'] = record['response']

In [103]:
# List the records to be deleted
numRecords = len(custom_dataset.records)
records_to_delete = list(custom_dataset.records[:numRecords])
# Delete the list of records from the dataset
custom_dataset.delete_records(records_to_delete)

In [104]:
custom_dataset.add_records(records)

Output()

-------------

